### Installing all necessary dependencies

In [ ]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!python -m spacy download es_core_news_lg

### Setup the model

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

### Setting up the LoRa Adapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

### Preprocessing the dataset of news


In [ ]:
import spacy
from datasets import load_dataset

In [ ]:
nlp = spacy.load("es_core_news_lg")

In [ ]:
def extract_first_loc(text):
    doc = nlp(text)
    first_loc = next((ent.text for ent in doc.ents if ent.label_ == "LOC"), None) # Find the first entity of type LOC

    return first_loc

In [ ]:
alpaca_prompt = """Tu tarea es escribir artículos de noticia que contengan siempre una fecha, un lugar y un acontecimiento. No puedes inventar información que no se te da, utiliza lenguaje formal.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = "Crea un artículo de noticias con esta información:"
    inputs = []
    outputs = []
    for contenido, resumen, fecha in zip(examples["Contenido"], examples["resumen"], examples["Fecha"]):
        location = extract_first_loc(contenido)
        if location:
            input_text = resumen + ". " + "Fecha: " + fecha + ". Lugar: " + location
            output_text = "El " + fecha +  ", en " + location + ". " + contenido
        else:
            input_text = resumen + ". " + "Fecha: " + fecha
            output_text = "El " + fecha + ". " + contenido
        print(input_text)
        print(output_text)
        inputs.append(input_text)
        outputs.append(output_text)
    print(len(inputs))
    print(len(outputs))
    texts = []

    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instructions, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [ ]:
dataset = load_dataset("BrauuHdzM/Noticias-con-resumen", split = "train")

In [ ]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

### Training


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

### Push the model adapters and tokenizer to the repository in HuggingFace

In [ ]:
model.push_to_hub(" ", token = "")  # Your repository and token in HuggingFace
tokenizer.push_to_hub("BrauuHdzM/llama3_noticias_espanol", token = "") # Your repository and token in HuggingFace